In [2]:
import numpy as np


In [63]:
#hw1
def compare(a,b, match=1, mismatch=-1):
    if(a==b):
        return match
    return mismatch

def n_w(seq1, seq2, match =1, mismatch=-1, gap = -1):
    matrix_weights = np.zeros((len(seq1)+1, len(seq2)+1))
    matrix_weights[0, :] = np.arange(0, -len(seq2)-1, -1)
    matrix_weights[:, 0] = np.arange(0, -len(seq1)-1, -1)
    for i in range( 1,matrix_weights.shape[0]):
        for j in range(1 ,matrix_weights.shape[1]):
      
            if(seq1[i-1]==seq2[j-1]):
                s1 = matrix_weights[i-1][j-1]+match
            else:
                s1 = matrix_weights[i-1][j-1]+mismatch
            s2 = matrix_weights[i][j-1]+gap
            s3 = matrix_weights[i-1][j]+gap
            matrix_weights[i][j] = max(s1,s2,s3)
    seq_alignment1 = ""
    seq_alignment2 = ""
    i = len(seq1)
    j = len(seq2)
    while(i>0 or j>0):
 
        #print(i,j)
        if (  (i>0 and j>0) and  matrix_weights[i][j] == matrix_weights[i-1][j-1]+ compare(seq1[i-1], seq2[j-1],match, mismatch ) ):
        #print(i, j)
            seq_alignment1  = seq1[i-1]+seq_alignment1 
            seq_alignment2  = seq2[j-1]+seq_alignment2 
        #print(seq_alignment2, seq_alignment1)
            i-=1
            j-=1
    
        elif ( i>0 and matrix_weights[i][j] == matrix_weights[i-1][j]+ gap):
        
            seq_alignment1  = seq1[i-1]+seq_alignment1 
            seq_alignment2  = "-"+seq_alignment2   
            i-=1
        else:
        
            seq_alignment1  = "-"+seq_alignment1 
            seq_alignment2  = seq2[j-1]+seq_alignment2         
            j=j-1
      
    #print(seq_alignment1)
    #print(seq_alignment2)
    #print(matrix_weights)
    return seq_alignment1, seq_alignment2

In [64]:
seq1="PEEMSVTSLDLTGGLPEATTPESEEAFTLPLLNDPEPKPSLEPVKNISNMELKAEPFD"
seq2="PEEMSVASLDLTGGLPEASTPESEEAFTLPLLNDPEPKPSLEPVKSISNVELKAEPFD"
seq3="SEELAAATALDLGAPSPAAAEEAFALPLMTEAPPAVPPKEPSGSGLELKAEPFD"
seq4="PGPGPLAEVRDLPGSTSAKEDGFGWLLPPPPPPPLPFQ"
seq5="PGPGPLAEVRDLPGSAPAKEDGFSWLLPPPPPPPLPFQ"

args = [seq1, seq2, seq3, seq4, seq5]

In [65]:
def entropy(list_input):
    unique_base = set(list_input)
    entropy_list = []    
    for base in unique_base:
        n_i = list_input.count(base)  
        P_i = n_i/len(list_input)  
        entropy_i = P_i*(np.log(P_i)/np.log(2))
        entropy_list.append(entropy_i)

    entropy = -(np.sum(entropy_list))

    return entropy

In [77]:
#es = np.nan((len(args), len(args)))
def nans(shape, dtype=float):
    a = np.empty(shape, dtype)
    a.fill(np.nan)
    return a

identity_matrix =nans([len(args),len(args)])
for i in range(len(args)):
    for j in range(i+1, len(args)):
        #print(j)
        
        seq_alignment1, seq_alignment2 = n_w(args[j], args[i])
        #print(len(seq_alignment1), len(seq_alignment2))
        count=0
        for e in range(len(seq_alignment1)):
            #print(e, end=" ")
            if(seq_alignment1[e]==seq_alignment2[e]):
                count=count+1
        
        identity_matrix [j][i]= count/(len(seq_alignment1))
        
        
identity_matrix 
 

array([[       nan,        nan,        nan,        nan,        nan],
       [0.93103448,        nan,        nan,        nan,        nan],
       [0.48387097, 0.51612903,        nan,        nan,        nan],
       [0.27586207, 0.30508475, 0.32142857,        nan,        nan],
       [0.28813559, 0.30508475, 0.33333333, 0.92105263,        nan]])

In [81]:
def dist(k,l,M):
    if k < l:
        return(M[k,l])
    else:
        return(M[l,k])
    
def upd_ind(i,j,index,d):
    a = index.pop(i)
    if type(a) == int:
        aa = a
    else:
        aa = a.copy()
    da = 0
    b = index.pop(j-1)
    if type(b) == int:
        bb = b
    else:
        bb = b.copy()
    db = 0
    while(type(aa) != int):
       
        aa = aa.pop()
        da += aa[1]
        aa = aa[0]
    while(type(bb) != int):
      
        bb = bb.pop(0)
       
        db += bb[1]
        bb = bb[0]
    
    a = [a, d/2 - da]
    b = [b, d/2 - db]
    index.append([a,b]) 
    return(index)


def create_ind(n):
    res = []
    for i in range(n):
        res.append(i)
    return(res)

def WPGMA( M):
    index = create_ind(M.shape[0])
    step = 1
    while M.shape[0] > 2:
  
        ind = np.unravel_index(np.nanargmin(M), M.shape)

        N = np.delete(M, [ind[0],ind[1]], 0)
        N = np.delete(N, [ind[0],ind[1]], 1)

        a = np.zeros(N.shape[0]+1)
        a.fill(np.nan)
        b = np.zeros((N.shape[0],1))
        N = np.hstack((N,b))
        N = np.vstack((N,a))

        nxt = 0
        for i in range(M.shape[0]):
            if i == ind[0] or i == ind[1]:
                continue
  
            N[nxt,N.shape[1]-1] =(dist(i,ind[0],M) + dist(i, ind[1], M))/2
            nxt += 1
     
        upd_ind(ind[0], ind[1],index, dist(ind[0], ind[1], M))
        M = N
        step+=1
    upd_ind(0, 1,index, dist(0, 1, M))

    return(index)

In [92]:
#tree= np.array(WPGMA(identity_matrix.T ))

#tsize = tree.shape
#tsize

(1, 2, 2)

In [109]:
tree

array([[[list([[1, 0.15254237288135594], [4, 0.15254237288135594]]),
         0.10647213865010743],
        [list([[2, 0.20132488479262672], [[[0, 0.13793103448275862], [3, 0.13793103448275862]], 0.0633938503098681]]),
         0.05768962673883665]]], dtype=object)